In [ ]:
%env LANGCHAIN_PROJECT=rag_book_03

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.vectorstores import Chroma
import os
from dotenv import load_dotenv

load_dotenv()

model = AzureChatOpenAI(
            azure_deployment=os.getenv("OPENAI_CHAT_DEPLOYMENT_NAME"),
            openai_api_type="azure",
            temperature=0.0,
        )

embeddings = AzureOpenAIEmbeddings(
            deployment=os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
            chunk_size=1,
            embedding_ctx_length=1000,)

In [ ]:
CHROMA_PERSISTENT_DIR = "../../data/chroma"
CHROMA_COLLECTION_NAME = "book_eng"

vectorstore = Chroma(
            collection_name=CHROMA_COLLECTION_NAME,
            persist_directory=CHROMA_PERSISTENT_DIR,
            embedding_function=embeddings,
        )


In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain.schema import format_document
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 3, "score_threshold": 0.9})

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [ ]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | model,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [ ]:
inputs = {"question": "What's the name of the singer?"}
result = final_chain.invoke(inputs)
result

### eval

In [ ]:
from langsmith import Client

client = Client()
dataset_name = "test"


In [ ]:
from langsmith.evaluation import RunEvaluator, EvaluationResult
from langchain.evaluation import load_evaluator

class BookFaithfulnessEvaluator(RunEvaluator):

    def __init__(self):
        self.evaluator = load_evaluator(
            "labeled_score_string", 
            criteria={"faithful": "How faithful is the submission to the reference context?"},
            normalize_by=10,
            llm=model,
        )

    def evaluate_run(self, run, example) -> EvaluationResult:
        res = self.evaluator.evaluate_strings(
            prediction=next(iter(run.outputs.values())),
            input=run.inputs["question"],
            # We are treating the documents as the reference context in this case.
            reference=example.inputs["docs"],
        )
        return EvaluationResult(key="book:faithful", **res)

In [ ]:
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    eval_llm=model,
    evaluators=[
        RunEvalConfig.QA(reference_key="answer", prediction_key="answer"),
        RunEvalConfig.LabeledCriteria("correctness", reference_key="answer", prediction_key="answer"), 

    ],
    custom_evaluators=[BookFaithfulnessEvaluator()],
    input_key="question",
)
results = client.run_on_dataset(
    llm_or_chain_factory=final_chain,
    dataset_name=dataset_name,
    evaluation=eval_config,
)   